# Reproducing figures from 12.5 year stochastic paper

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

import os, glob, json, pickle
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as sl

import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
import enterprise.constants as const

import corner
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc

import enterprise_extensions
from enterprise_extensions import models, model_utils, hypermodel

## Download chains from 12.5 year stochastic paper
* The following cells make sure that a folder exists to put the files in,
* then they download the chains from the NANOGrav server.

In [2]:
fpath = '../../chains/downloaded_chains/'
if not os.path.exists(fpath):
    os.mkdir(fpath)

## model_2a, 5 frequency power law, gamma=4.33

In [ ]:
# enterprise_extensions (e_e) code to reproduce model:
# models.model_2a(psrs, psd='powerlaw', noisedict=params, n_gwbfreqs=5,
#                 gamma_common=4.33)

In [ ]:
# download params:
!curl https://data.nanograv.org/static/data/12p5yr_chains/12p5yr_DE438_model2a_cRN5freq_gammaFixed4.33_pars.txt -o ../../chains/downloaded_chains/12p5yr_DE438_model2a_cRN5freq_gammaFixed4.33_chain.pars.txt

# download chain:
!curl https://data.nanograv.org/static/data/12p5yr_chains/12p5yr_DE438_model2a_cRN5freq_gammaFixed4.33_chain.gz -o ../../chains/downloaded_chains/12p5yr_DE438_model2a_cRN5freq_gammaFixed4.33_chain.gz

## model_2a, 5 frequency power law, gamma vary

In [ ]:
# e_e code to reproduce model:
# models.model_2a(psrs, psd='powerlaw', noisedict=params, n_gwbfreqs=5)

In [2]:
# download params:
!curl https://data.nanograv.org/static/data/12p5yr_chains/12p5yr_DE438_model2a_cRN5freq_gammaVary_chain_pars.txt -o ../../chains/downloaded_chains/12p5yr_DE438_model2a_cRN5freq_gammaVary_chain_pars.txt

# download chain:
!curl https://data.nanograv.org/static/data/12p5yr_chains/12p5yr_DE438_model2a_cRN5freq_gammaVary_chain.gz -o ../../chains/downloaded_chains/12p5yr_DE438_model2a_cRN5freq_gammaVary_chain.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   168  100   168    0     0    268      0 --:--:-- --:--:-- --:--:--   268
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  128M  100  128M    0     0  8560k      0  0:00:15  0:00:15 --:--:-- 11.2M


In [1]:




!curl https://data.nanograv.org/static/data/12p5yr_chains/12p5yr_DE438_model2a_cRN30freq_gammaVary_chain.gz -o ../../chains/downloaded_chains/12p5yr_DE438_model2a_cRN30freq_gammaVary_chain.gz

!curl https://data.nanograv.org/static/data/12p5yr_chains/12p5yr_DE438_model2a_PSDbrokenplaw_gammaVary_chain.gz -o ../../chains/downloaded_chains/12p5yr_DE438_model2a_PSDbrokenplaw_gammaVary_chain.gz

!curl https://data.nanograv.org/static/data/12p5yr_chains/12p5yr_DE438_model2a_PSDspectrum_chain.gz -o ../../chains/downloaded_chains/12p5yr_DE438_model2a_PSDspectrum_chain.gz

!curl https://data.nanograv.org/static/data/12p5yr_chains/12p5yr_DE438wBayesEphem_model2a_cRN5freq_gammaFixed4.33_chain.gz -o ../../chains/downloaded_chains/12p5yr_DE438wBayesEphem_model2a_cRN5freq_gammaFixed4.33_chain.gz

!curl https://data.nanograv.org/static/data/12p5yr_chains/12p5yr_INPOP19_model2a_cRN5freq_gammaFixed4.33_chain.gz -o ../../chains/downloaded_chains/12p5yr_INPOP19_model2a_cRN5freq_gammaFixed4.33_chain.gz

!curl https://data.nanograv.org/static/data/12p5yr_chains/12p5yr_INPOP19wBayesEphem_model2a_cRN5freq_gammaFixed4.33_chain.gz -o ../../chains/downloaded_chains/12p5yr_INPOP19wBayesEphem_model2a_cRN5freq_gammaFixed4.33_chain.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2519  100  2519    0     0   5817      0 --:--:-- --:--:-- --:--:--  5831
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  177M  100  177M    0     0  9360k      0  0:00:19  0:00:19 --:--:-- 12.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  128M  100  128M    0     0  8480k      0  0:00:15  0:00:15 --:--:-- 10.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  147M  100  147M    0     0  8559k      0  0:00:17  0:00:17 --:--:-- 10.9M
  % Total    % Received % Xferd  Average Speed   Tim

In [6]:
chain_DE438_bplaw_vary = np.loadtxt('../../chains/downloaded_chains/12p5yr_DE438_model2a_PSDbrokenplaw_gammaVary_chain.gz', usecols=np.arange(-7,-3), skiprows=25000)
chain_DE438_bplaw_vary = chain_DE438_bplaw_vary[::5]
pars_DE438_bplaw_vary = np.loadtxt('../../chains/data/params_txt/12p5yr_DE438_model2a_PSDbrokenplaw_gammaVary_pars.txt', dtype=np.str)

## 5 frequency
# models.model_2a(psrs, psd='powerlaw', noisedict=params, components=5,
#              n_rnfreqs = None, n_gwbfreqs=None, gamma_common=None,
#              delta_common=None, upper_limit=False, bayesephem=False,
#              be_type='orbel', white_vary=False, is_wideband=False,
#              use_dmdata=False, select='backend',
#              pshift=False, pseed=None, psr_models=False)

chain_DE438_vary = np.loadtxt('../../chains/downloaded_chains/12p5yr_DE438_model2a_cRN5freq_gammaVary_chain.gz', usecols=[90,91,92], skiprows=25000)
chain_DE438_vary = chain_DE438_vary[::4]

OSError: ../../chains/data/params_txt/12p5yr_DE438_model2a_PSDbrokenplaw_gammaVary_pars.txt not found.